<a href="https://colab.research.google.com/github/manuelrucci7/deep-learning-course/blob/main/colab/LLMCustomTrainingTextClassificationTransformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Custom Traning

In [36]:
%%capture
!pip install transformers datasets evaluate

## Dataset

Il problema consiste nel creare un dataset con 200 frasi in italiano, suddivise in due classi: "Rischio" (etichetta 1) e "Non Rischio" (etichetta 0). Le frasi devono essere costruite in base a un contesto economico, dove un aumento del tasso di interesse implica una situazione di rischio economico, mentre una diminuzione del tasso di interesse rappresenta un contesto più favorevole e meno rischioso. Il dataset deve essere bilanciato, con 100 frasi per ciascuna classe, e strutturato in modo tale da includere frasi pertinenti all'argomento. Questo dataset può essere utilizzato per addestrare modelli di classificazione o analisi economica.

In [93]:
from datasets import DatasetDict, Dataset

# Generazione di 110 frasi (55 Rischio, 55 Non Rischio)
data = {
    "sentence": [
        # Frasi Rischio (etichetta 1)
        "L'aumento del tasso di interesse rallenta l'economia.",
        "Un forte aumento del tasso di interesse può portare alla recessione.",
        "La banca centrale ha alzato i tassi, aumentando il rischio per le imprese.",
        "L'aumento improvviso dei tassi di interesse crea incertezza nei mercati.",
        "Con tassi in aumento, le aziende potrebbero ridurre gli investimenti.",
        "Tassi di interesse elevati aumentano il rischio di default sui prestiti.",
        "L'incremento dei tassi rende il debito pubblico più oneroso.",
        "Un rialzo repentino dei tassi spaventa gli investitori.",
        "L'aumento dei tassi frena la crescita economica globale.",
        "Tassi più alti scoraggiano i consumatori dal contrarre nuovi debiti.",
        "Un rialzo dei tassi crea instabilità nei settori ad alta leva finanziaria.",
        "Con i tassi in crescita, i mutui diventano meno accessibili.",
        "Tassi elevati aumentano i rischi di una contrazione economica.",
        "L'aumento dei tassi rende più costosi i prestiti per i consumatori.",
        "Con tassi più alti, il rischio di insolvenza aumenta per le famiglie.",
        "Un rialzo dei tassi penalizza il settore del credito al consumo.",
        "Tassi elevati creano tensioni nel settore immobiliare.",
        "L'aumento dei tassi rallenta il flusso di capitali nelle economie emergenti.",
        "Con tassi al rialzo, le banche riducono l'erogazione di prestiti.",
        "Un improvviso aumento dei tassi destabilizza le economie fragili.",
        "L'aumento dei tassi riduce la liquidità nei mercati emergenti.",
        "Tassi alti scoraggiano nuovi progetti di investimento.",
        "La crescita dei tassi rende i finanziamenti aziendali meno sostenibili.",
        "Un incremento del tasso porta a una contrazione nei consumi.",
        "Con l'aumento dei tassi, le obbligazioni diventano meno attraenti.",
        # Aggiungi altre frasi "Rischio" fino a raggiungere 100
        "Tassi di interesse elevati aumentano i costi di finanziamento per le imprese.",
        "L'aumento dei tassi riduce la domanda di mutui da parte delle famiglie.",
        "Con tassi più alti, i mercati azionari tendono a subire una flessione.",
        "Tassi elevati ostacolano la ripresa economica in tempi di crisi.",
        "Un rialzo dei tassi scoraggia gli investimenti nel settore tecnologico.",
        "Con tassi in aumento, molte aziende rivedono al ribasso i loro budget.",
        "L'incremento dei tassi peggiora la situazione finanziaria dei paesi in via di sviluppo.",
        "Un aumento repentino dei tassi di interesse può causare una fuga di capitali.",
        "Con i tassi elevati, i consumatori sono meno propensi a spendere.",
        "Tassi di interesse alti limitano l'accesso al credito per le piccole imprese.",
        "L'aumento dei tassi di interesse penalizza il settore immobiliare.",
        "Un incremento dei tassi crea instabilità nei settori ad alto debito.",
        "Tassi più alti riducono la redditività degli investimenti a lungo termine.",
        "L'aumento dei tassi frena l'innovazione nei settori chiave.",
        "Un rialzo dei tassi aumenta la pressione sui bilanci statali già in deficit.",
        "Tassi elevati rendono più difficile il finanziamento di progetti infrastrutturali.",
        "Con tassi alti, le esportazioni diventano meno competitive.",
        "Un improvviso aumento dei tassi destabilizza le economie emergenti.",
        "L'incremento dei tassi riduce i margini di profitto delle imprese manifatturiere.",
        "Tassi di interesse elevati portano a un calo della fiducia degli investitori.",
        "Con tassi più alti, le imprese tagliano le spese per la formazione dei dipendenti.",
        "Un forte rialzo dei tassi aumenta il rischio di insolvenza per le famiglie.",
        "L'aumento dei tassi riduce la domanda di beni durevoli.",
        "Tassi elevati ostacolano il flusso di capitali verso le startup innovative.",
        "Con l'aumento dei tassi, il settore turistico subisce un calo di investimenti.",
        "Un incremento dei tassi limita l'espansione delle aziende nei mercati esteri.",
        "Tassi elevati aumentano le difficoltà di accesso al credito per i giovani imprenditori.",
        "L'aumento dei tassi di interesse aggrava le difficoltà finanziarie delle imprese agricole.",
        "Con tassi elevati, il debito diventa insostenibile per molte economie fragili.",
        "Un rialzo improvviso dei tassi di interesse può causare un'ondata di fallimenti aziendali."
    ] + [
        # Frasi Non Rischio (etichetta 0)
        "La diminuzione del tasso di interesse favorisce gli investimenti.",
        "Un calo del tasso di interesse stimola il mercato immobiliare.",
        "Il taglio dei tassi di interesse incoraggia il consumo.",
        "La riduzione graduale dei tassi rende i prestiti più accessibili.",
        "La politica di tassi bassi favorisce la crescita economica.",
        "Un abbassamento del tasso d'interesse migliora la fiducia dei consumatori.",
        "La diminuzione dei tassi riduce il costo del capitale per le aziende.",
        "Tassi bassi stimolano il mercato dei prestiti personali.",
        "Un calo dei tassi sostiene le piccole e medie imprese.",
        "Tassi di interesse più bassi migliorano la competitività delle esportazioni.",
        "La riduzione dei tassi aumenta la liquidità nel sistema economico.",
        "Un abbassamento dei tassi di interesse supporta il settore bancario.",
        "Un calo del tasso d'interesse stimola la domanda di beni durevoli.",
        "Un taglio ai tassi di interesse favorisce il settore tecnologico.",
        "Tassi più bassi incentivano gli investimenti nelle infrastrutture.",
        "La politica di tassi bassi rafforza il mercato azionario.",
        "La diminuzione dei tassi riduce il rischio di crisi finanziaria.",
        "Tassi bassi incoraggiano gli imprenditori a sviluppare nuove attività.",
        "La diminuzione dei tassi stimola la ripresa economica.",
        "Tassi bassi sostengono la crescita del settore manifatturiero.",
        "Un calo dei tassi favorisce gli investimenti esteri.",
        "La diminuzione dei tassi facilita l'accesso al credito per i consumatori.",
        "La politica di tassi bassi rende i mutui più accessibili.",
        "La riduzione dei tassi sostiene il settore agricolo.",
        "Con i tassi bassi, l'economia si espande più rapidamente.",
        # Aggiungi altre frasi "Non Rischio" fino a raggiungere 100
        "Un abbassamento dei tassi di interesse promuove la stabilità economica.",
        "Tassi bassi favoriscono il finanziamento di progetti innovativi.",
        "La riduzione dei tassi rende i prestiti aziendali più sostenibili.",
        "Tassi di interesse bassi stimolano l'acquisto di nuove abitazioni.",
        "La diminuzione dei tassi facilita la creazione di nuove imprese.",
        "Con tassi bassi, il settore turistico beneficia di maggiori investimenti.",
        "Tassi bassi migliorano la competitività delle economie locali.",
        "Un calo dei tassi di interesse rende il debito pubblico più gestibile.",
        "La riduzione dei tassi promuove l'accessibilità ai prestiti per le famiglie.",
        "Tassi bassi aumentano la fiducia dei consumatori nei mercati finanziari.",
        "Un abbassamento dei tassi incoraggia l'espansione delle PMI.",
        "La diminuzione dei tassi stimola l'acquisto di beni di consumo durevoli.",
        "Tassi bassi migliorano il flusso di capitali verso i paesi in via di sviluppo.",
        "Con i tassi bassi, i settori tecnologici ricevono maggiore supporto finanziario.",
        "Un calo dei tassi di interesse favorisce la creazione di posti di lavoro.",
        "La riduzione dei tassi aiuta le economie a riprendersi dalle crisi.",
        "Con tassi bassi, i giovani imprenditori possono accedere più facilmente al credito.",
        "Tassi bassi aumentano la capacità di spesa delle famiglie.",
        "Un abbassamento dei tassi incentiva gli investimenti nel settore energetico.",
        "Con tassi bassi, le economie emergenti attraggono più investimenti esteri.",
        "La riduzione dei tassi di interesse migliora il benessere economico generale.",
        "Un calo dei tassi rafforza la stabilità delle banche centrali.",
        "Con tassi bassi, i mercati azionari registrano una crescita sostenuta.",
        "Un abbassamento dei tassi riduce i costi di finanziamento per le imprese.",
        "Tassi bassi favoriscono la ripresa del mercato immobiliare.",
        "La riduzione dei tassi aumenta la liquidità per le economie locali.",
        "Un calo dei tassi rende più semplice finanziare progetti infrastrutturali.",
        "Tassi bassi stimolano la domanda nei settori legati alla produzione industriale.",
        "Con tassi bassi, le economie fragili possono riprendersi più rapidamente.",
        "La diminuzione dei tassi sostiene gli investimenti nei settori ad alta innovazione."
    ],
    "label": [1] * 55 + [0] * 55,  # 55 Rischio (1) e 55 Non Rischio (0)
    "idx": list(range(110))  # Indici unici per ciascuna frase
}


In [94]:
# Creare il Dataset da addestramento e validazione
dataset = Dataset.from_dict(data)

# Dividere il dataset train e  test
split = dataset.train_test_split(test_size=0.2, seed=42)

# DatasetDict
raw_datasets = DatasetDict({
    "train": split["train"],
    "validation": split["test"]
})

# Visualizza la struttura finale
print(raw_datasets)

DatasetDict({
    train: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 88
    })
    validation: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 22
    })
})


## Model Tokenizer

In [95]:
#def compute_metrics(eval_preds):
#    metric = evaluate.load("glue", "mrpc")
#    logits, labels = eval_preds
#    predictions = np.argmax(logits, axis=-1)
#    return metric.compute(predictions=predictions, references=labels)

def tokenize_function(example):
    return tokenizer(example["sentence"], truncation=True)


In [96]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer, DataCollatorWithPadding
import evaluate
import numpy as np

#checkpoint = "bert-base-uncased"
checkpoint = "tabularisai/multilingual-sentiment-analysis"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2, ignore_mismatched_sizes=True)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at tabularisai/multilingual-sentiment-analysis and are newly initialized because the shapes did not match:
- classifier.bias: found shape torch.Size([5]) in the checkpoint and torch.Size([2]) in the model instantiated
- classifier.weight: found shape torch.Size([5, 768]) in the checkpoint and torch.Size([2, 768]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [97]:
tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

Map:   0%|          | 0/88 [00:00<?, ? examples/s]

Map:   0%|          | 0/22 [00:00<?, ? examples/s]

In [99]:
import numpy as np
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def compute_metrics(eval_preds):
    """
    Compute metrics for evaluation.

    Args:
        eval_preds: A tuple (logits, labels) where logits are the model's outputs
                    and labels are the true labels.

    Returns:
        dict: A dictionary with the evaluation metrics (accuracy, precision, recall, F1 score).
    """
    # Unpack predictions and labels
    logits, labels = eval_preds

    # Convert logits to predicted class labels
    predictions = np.argmax(logits, axis=-1)

    # Calculate accuracy, precision, recall, and F1 score
    accuracy = accuracy_score(labels, predictions)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average="binary")

    # Return metrics as a dictionary
    return {
        "accuracy": accuracy,
        "precision": precision,
        "recall": recall,
        "f1": f1
    }

In [100]:
training_args = TrainingArguments("test-trainer", evaluation_strategy="epoch", num_train_epochs=4)
# training_args = TrainingArguments(
#     output_dir="test-trainer",          # Output directory
#     evaluation_strategy="epoch",    # Evaluate after each epoch
#     #learning_rate=2e-5,             # Learning rate
#     #per_device_train_batch_size=16, # Batch size for training
#     #per_device_eval_batch_size=16,  # Batch size for evaluation
#     num_train_epochs=3,             # Number of epochs
#     #weight_decay=0.01,              # Weight decay
#     #logging_dir="./logs",           # Directory for logs
#     #logging_steps=10,               # Log every 10 steps
#     #save_total_limit=2,             # Save only the last 2 checkpoints
#     #save_steps=500,                 # Save every 500 steps
# )

trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-100-63bc3597193a>:16: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
import wandb
wandb.init(mode="disabled")
# https://huggingface.co/docs/datasets/v1.13.0/quickstart.html
# https://huggingface.co/docs/transformers/main_classes/trainer
# https://huggingface.co/learn/nlp-course/chapter7/3
#os.environ["WANDB_DISABLED"] = "true"
trainer.train()


Epoch,Training Loss,Validation Loss


## Validation

In [91]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import torch
import torch.nn.functional as F

model_path = "/content/test-trainer/checkpoint-15"  # Update this path to where your model is saved
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSequenceClassification.from_pretrained(model_path)

In [92]:
label = {
    0: "Non Rischio",
    1: "Rischio"
}

# Rischio
sentences = [
    "Con tassi elevati, il costo del debito aumenta per le famiglie.",
    "Un incremento dei tassi spinge le imprese a tagliare i costi.",
    "Con tassi in rialzo, le economie fragili subiscono pressioni aggiuntive.",
    "Tassi più alti portano a una diminuzione della fiducia dei consumatori.",
    "L'aumento dei tassi rende il credito al consumo meno conveniente.",
    "Con tassi elevati, le famiglie riducono la spesa discrezionale.",
    "Un forte aumento dei tassi peggiora le condizioni economiche generali."
]

# Frasi non rischio
"""
sentences= [
   "Con i tassi bassi, le famiglie possono investire in nuove proprietà.",
   "La riduzione dei tassi favorisce il settore delle esportazioni.",
   "Un calo dei tassi promuove il benessere delle economie emergenti.",
   "Tassi bassi aumentano la stabilità economica a livello globale.",
   "La diminuzione dei tassi riduce i rischi di una recessione prolungata.",
   "Con tassi bassi, le startup trovano più finanziamenti per crescere.",
   "Un abbassamento dei tassi rende più attraenti gli investimenti infrastrutturali."
]
"""

token_inputs = tokenizer(sentences, return_tensors="pt", padding=True, truncation=True)  # Ensure tensor shapes are correct
inputs = {k: v.to(model.device) for k, v in token_inputs.items()}
with torch.no_grad():
    outputs = model(**inputs)
logits = outputs.logits
probabilities = torch.softmax(logits, dim=-1)
for sentence, probs in zip(sentences, probabilities):
    print(f"{label[predicted_class.item()]}: {probs[predicted_class.item()]:.3f}: {sentence} {probs}")
    predicted_class = torch.argmax(probs, dim=-1)


Rischio: 0.965: Con tassi elevati, il costo del debito aumenta per le famiglie. tensor([0.0354, 0.9646])
Rischio: 0.960: Un incremento dei tassi spinge le imprese a tagliare i costi. tensor([0.0399, 0.9601])
Rischio: 0.961: Con tassi in rialzo, le economie fragili subiscono pressioni aggiuntive. tensor([0.0385, 0.9615])
Rischio: 0.966: Tassi più alti portano a una diminuzione della fiducia dei consumatori. tensor([0.0337, 0.9663])
Rischio: 0.966: L'aumento dei tassi rende il credito al consumo meno conveniente. tensor([0.0336, 0.9664])
Rischio: 0.964: Con tassi elevati, le famiglie riducono la spesa discrezionale. tensor([0.0356, 0.9644])
Rischio: 0.964: Un forte aumento dei tassi peggiora le condizioni economiche generali. tensor([0.0359, 0.9641])
